In [1]:
google_colab = False

# Google Colab用事前準備

In [2]:
# install
if google_colab:
  !pip install pybind11
  %cd /content
  !nvcc -V
  !sudo rm -fr BinaryBrain
  !rm -fr BinaryBrain
  !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
  %cd /content/BinaryBrain/python
  !sh copy_src.sh
  !python3 setup.py build
  !python3 setup.py develop

  import binarybrain as bb

In [3]:
import binarybrain as bb
import importlib
importlib.reload(bb)
importlib

<module 'importlib' from 'c:\\users\\ryuji\\appdata\\local\\programs\\python\\python37\\lib\\importlib\\__init__.py'>

In [4]:
# mount
if google_colab:
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python

In [5]:
import binarybrain as bb
prop = bb.get_device_properties()
print(prop)


name                     : GeForce GT 1030
totalGlobalMem           : 2147483648
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1468000
totalConstMem            : 65536
major                    : 6
minor                    : 1
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 3
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




# メインコード

In [6]:
import binarybrain as bb
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict


In [7]:
# load MNIST data
td = bb.load_mnist()

In [8]:
td['t_shape']

[10]

In [9]:
x = td['x_train']
t = td['t_train']

In [10]:
t_img = np.zeros((len(x), 28*28, 11))

In [11]:
i = 0
mask = x[i] > 0.5

In [12]:
t[i]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [13]:
t_img[i, mask, 0:10] = t[i]

In [14]:
# 結果改変
def make_t_image(x, t):
    t_img = np.zeros((len(x), 28*28, 11))
    for i in range(len(x)):
        mask0 = x[i] <= 0.5
        mask1 = x[i] > 0.5
        t_img[i, mask0, 10] = 1.0
        t_img[i, mask1, 0:10] = t[i]
    return t_img.reshape(-1, 28*28*11)

td['t_train'] = make_t_image(td['x_train'], td['t_train'])
td['t_test']  = make_t_image(td['x_test'], td['t_test'])
td['t_shape'] = [28*28*11]

## 元レイヤー

In [15]:
modulation_size = 8
layer_rel2bin = bb.RealToBinary.create(modulation_size, framewise=True)
layer_bin2rel = bb.BinaryToReal.create(modulation_size)

# create layer
layer0_affine  = bb.DenseAffine.create([36])
layer0_norm    = bb.BatchNormalization.create()
layer0_act     = bb.ReLU.create()
#layer0_act     = bb.Binarize.create()
layer1_affine  = bb.DenseAffine.create([36])
layer1_norm    = bb.BatchNormalization.create()
layer1_act     = bb.ReLU.create()
layer2_affine  = bb.DenseAffine.create([36])
layer2_norm    = bb.BatchNormalization.create()
layer2_act     = bb.ReLU.create()
layer3_affine  = bb.DenseAffine.create([36])
layer3_norm    = bb.BatchNormalization.create()
layer3_act     = bb.ReLU.create()
layer4_affine  = bb.DenseAffine.create([36])
layer4_norm    = bb.BatchNormalization.create()
layer4_act     = bb.ReLU.create()
layer5_affine  = bb.DenseAffine.create([36])
layer5_norm    = bb.BatchNormalization.create()
layer5_act     = bb.ReLU.create()
layer6_affine  = bb.DenseAffine.create([11])
layer6_norm    = bb.BatchNormalization.create()
layer6_act     = bb.ReLU.create()

# main network
cnv0_sub = bb.Sequential.create()
cnv0_sub.add(layer0_affine)
cnv0_sub.add(layer0_norm)
cnv0_sub.add(layer0_act)
layer0_cnv = bb.LoweringConvolution.create(cnv0_sub, 3, 3, 1, 1, 'same')

cnv1_sub = bb.Sequential.create()
cnv1_sub.add(layer1_affine)
cnv1_sub.add(layer1_norm)
cnv1_sub.add(layer1_act)
layer1_cnv = bb.LoweringConvolution.create(cnv1_sub, 3, 3, 1, 1, 'same')

cnv2_sub = bb.Sequential.create()
cnv2_sub.add(layer2_affine)
cnv2_sub.add(layer2_norm)
cnv2_sub.add(layer2_act)
layer2_cnv = bb.LoweringConvolution.create(cnv2_sub, 3, 3, 1, 1, 'same')

cnv3_sub = bb.Sequential.create()
cnv3_sub.add(layer3_affine)
cnv3_sub.add(layer3_norm)
cnv3_sub.add(layer3_act)
layer3_cnv = bb.LoweringConvolution.create(cnv3_sub, 3, 3, 1, 1, 'same')

cnv4_sub = bb.Sequential.create()
cnv4_sub.add(layer4_affine)
cnv4_sub.add(layer4_norm)
cnv4_sub.add(layer4_act)
layer4_cnv = bb.LoweringConvolution.create(cnv4_sub, 3, 3, 1, 1, 'same')

cnv5_sub = bb.Sequential.create()
cnv5_sub.add(layer5_affine)
cnv5_sub.add(layer5_norm)
cnv5_sub.add(layer5_act)
layer5_cnv = bb.LoweringConvolution.create(cnv5_sub, 3, 3, 1, 1, 'same')

cnv6_sub = bb.Sequential.create()
cnv6_sub.add(layer6_affine)
cnv6_sub.add(layer6_norm)
cnv6_sub.add(layer6_act)
layer6_cnv = bb.LoweringConvolution.create(cnv6_sub, 3, 3, 1, 1, 'same')

main_net = bb.Sequential.create()

#main_net.add(layer_rel2bin)
main_net.add(layer0_cnv)
main_net.add(layer1_cnv)
#main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer2_cnv)
main_net.add(layer3_cnv)
#main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer4_cnv)
main_net.add(layer5_cnv)
main_net.add(layer6_cnv)
#main_net.add(layer_bin2rel)

main_net.set_input_shape(td['x_shape'])

# Load
net_path = 'cifar10-dense-cnn'
os.makedirs(net_path, exist_ok=True)

if False:
    layer0_affine.load_json(os.path.join(net_path, 'layer0_affine.json'))
    layer1_affine.load_json(os.path.join(net_path, 'layer1_affine.json'))
    layer2_affine.load_json(os.path.join(net_path, 'layer2_affine.json'))
    layer3_affine.load_json(os.path.join(net_path, 'layer3_affine.json'))
    layer4_affine.load_json(os.path.join(net_path, 'layer4_affine.json'))
    layer5_affine.load_json(os.path.join(net_path, 'layer5_affine.json'))

    layer0_norm.load_json(os.path.join(net_path, 'layer0_norm.json'))
    layer1_norm.load_json(os.path.join(net_path, 'layer1_norm.json'))
    layer2_norm.load_json(os.path.join(net_path, 'layer2_norm.json'))
    layer3_norm.load_json(os.path.join(net_path, 'layer3_norm.json'))
    layer4_norm.load_json(os.path.join(net_path, 'layer4_norm.json'))
    layer5_norm.load_json(os.path.join(net_path, 'layer5_norm.json'))

main_net.send_command('parameter_lock false')
layer0_cnv.send_command('parameter_lock false')
layer1_cnv.send_command('parameter_lock false')

#loss      = bb.LossSoftmaxCrossEntropy.create()
#metrics   = bb.MetricsCategoricalAccuracy.create()
loss      = bb.LossMeanSquaredError.create()
metrics   = bb.MetricsMeanSquaredError.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(main_net.get_parameters(), main_net.get_gradients())

In [16]:
print(main_net.get_info())

----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {28, 28, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [DenseAffine] 
       input  shape : {3, 3, 1}       output shape : {36}
      ----------------------------------------------------------------
      [BatchNormalization] 
       input  shape : {36}       output shape : {36}
      ----------------------------------------------------------------
      [ReLU] 
       input  shape : {36}       output shape : {36}
    ---------------------------------------------

In [17]:
runner = bb.Runner(main_net, "mnist-segment-dense-cnn", loss, metrics, optimizer)
runner.fitting(td, epoch_size=8, mini_batch_size=32, file_write=False, file_read=False, init_eval=False)

epoch=1 test_MSE=0.083042 test_loss=0.083042 train_MSE=0.083012 train_loss=0.083012


epoch=2 test_MSE=0.082770 test_loss=0.082770 train_MSE=0.082756 train_loss=0.082756


epoch=3 test_MSE=0.082769 test_loss=0.082769 train_MSE=0.082755 train_loss=0.082755


epoch=4 test_MSE=0.082584 test_loss=0.082584 train_MSE=0.082576 train_loss=0.082576


epoch=5 test_MSE=0.082226 test_loss=0.082226 train_MSE=0.082234 train_loss=0.082234


epoch=6 test_MSE=0.081822 test_loss=0.081822 train_MSE=0.081841 train_loss=0.081841


epoch=7 test_MSE=0.081569 test_loss=0.081569 train_MSE=0.081594 train_loss=0.081594


epoch=8 test_MSE=0.081239 test_loss=0.081239 train_MSE=0.081264 train_loss=0.081264


In [18]:
if True:
    layer0_affine.save_json(os.path.join(net_path, 'layer0_affine.json'))
    layer1_affine.save_json(os.path.join(net_path, 'layer1_affine.json'))
    layer2_affine.save_json(os.path.join(net_path, 'layer2_affine.json'))
    layer3_affine.save_json(os.path.join(net_path, 'layer3_affine.json'))
    layer4_affine.save_json(os.path.join(net_path, 'layer4_affine.json'))
    layer5_affine.save_json(os.path.join(net_path, 'layer5_affine.json'))

    layer0_norm.save_json(os.path.join(net_path, 'layer0_norm.json'))
    layer1_norm.save_json(os.path.join(net_path, 'layer1_norm.json'))
    layer2_norm.save_json(os.path.join(net_path, 'layer2_norm.json'))
    layer3_norm.save_json(os.path.join(net_path, 'layer3_norm.json'))
    layer4_norm.save_json(os.path.join(net_path, 'layer4_norm.json'))
    layer5_norm.save_json(os.path.join(net_path, 'layer5_norm.json'))

In [19]:
main_net.send_command('parameter_lock true')

modulation_size = 4
layer_rel2bin = bb.RealToBinary.create(modulation_size, framewise=True)
layer_bin2rel = bb.BinaryToReal.create(modulation_size)

net = bb.Sequential.create()
net.add(layer_rel2bin)
net.add(main_net)
net.add(layer_bin2rel)
net.set_input_shape(td['x_shape'])

loss      = bb.LossSoftmaxCrossEntropy.create()
metrics   = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(net.get_parameters(), net.get_gradients())

runner = bb.Runner(net, "mnist-dense-cnn-binary", loss, metrics, optimizer)
runner.fitting(td, epoch_size=2, mini_batch_size=32, file_write=False, file_read=False)

epoch=1 test_accuracy=0.270900 test_loss=9.055107 train_accuracy=0.263750 train_loss=9.055155


epoch=2 test_accuracy=0.270400 test_loss=9.056833 train_accuracy=0.264500 train_loss=9.056866
